In [9]:
import boto3
import json
import requests


s3 = boto3.client("s3")
secretsmanager = boto3.client("secretsmanager")


# class ImproperObjectNameError(Exception):
#     """
#     Raised when a bucket name cannot be inferred from an object name.
#     """

#     default_message = (
#         "Could not determine a bucket name from the object name. "
#         "Object name must be formatted as "
#         "'s3://<bucket-name>/path/to/object'."
#     )

#     def __init(self, message=default_messagee):
#         self.message = message
#         super().__init__(self.message)


# @dataclass(frozen=True)


def lambda_handler(event, context):
    sports_list = event.get("sports_list")
    regions_list = event.get("regions_list")
    API_KEY = json.loads(
        secretsmanager.get_secret_value(
            SecretId="odds_api_key"
        )["SecretString"]
    )["key"]

    response_list = []
    for sport in sports_list:
        payload = {
            "apiKey": API_KEY,
            "oddsFormat": "decimal",
            "regions": regions_list
        }
        
        response = requests.get(
            url=f"https://api.the-odds-api.com/v4/sports/{sport}/odds",
            params=payload
        )
        response_list += json.loads(response.content)
    s3.put_object(
        Bucket=event.get("s3_bucket"),
        Key=event.get("s3_key"),
        Body=json.dumps(response_list),
        ContentType="application/json"
    )
    return

In [10]:
test_event = {
    "sports_list": ["basketball_nba"],
    "regions_list": ["us", "us2"]
}

lambda_handler(test_event, {})

'[{"id": "373b7cbc2df4ed9d1e05cf322b1217fe", "sport_key": "basketball_nba", "sport_title": "NBA", "commence_time": "2025-03-02T20:40:00Z", "home_team": "Cleveland Cavaliers", "away_team": "Portland Trail Blazers", "bookmakers": [{"key": "draftkings", "title": "DraftKings", "last_update": "2025-03-02T23:25:09Z", "markets": [{"key": "h2h", "last_update": "2025-03-02T23:25:09Z", "outcomes": [{"name": "Cleveland Cavaliers", "price": 1.28}, {"name": "Portland Trail Blazers", "price": 3.6}]}]}, {"key": "fanduel", "title": "FanDuel", "last_update": "2025-03-02T23:25:11Z", "markets": [{"key": "h2h", "last_update": "2025-03-02T23:25:11Z", "outcomes": [{"name": "Cleveland Cavaliers", "price": 1.26}, {"name": "Portland Trail Blazers", "price": 3.8}]}]}, {"key": "betrivers", "title": "BetRivers", "last_update": "2025-03-02T23:25:09Z", "markets": [{"key": "h2h", "last_update": "2025-03-02T23:25:09Z", "outcomes": [{"name": "Cleveland Cavaliers", "price": 1.3}, {"name": "Portland Trail Blazers", "pri